### Get sample's filename

In [143]:
import pandas as pd
import pickle

In [144]:
def get_executable_sample() -> dict:
    df = pd.read_csv("../C malware info/sample_info.csv")
    family_lst = df.family.unique()
    sample_dct = {}
    for key in family_lst:
        sample_dct[key] = []

    for idx, row in df.iterrows():
        if row.platform in ['X86_64','X86','ARM']:
            sample_dct[row.family].append(row.filename)
    return sample_dct

sample_dct = get_executable_sample()

# save dictionary
with open('sample_dct.pkl', 'wb') as outp:
    pickle.dump(sample_dct, outp, pickle.HIGHEST_PROTOCOL)

In [145]:
# load dictionary
with open('sample_dct.pkl', 'rb') as inp:
    sample_dct = pickle.load(inp)

In [146]:
sample_dct

{'Xorddos': ['07c070b717a23453a2b71c7582d6a928.bin',
  '0aefb67c01a24d05351b093455203fa2.bin',
  '0bc90c333f08237475a08c7158aba345.bin',
  '127ee9c007774e8f1101fb40ea05b42e.bin',
  '16eb5d018d45024462dd63de470a8575.bin',
  '24dca1b2097ad6c2e5d8b03d3022f010.bin',
  '39b38284c6ca30d09481fcb5dc7197aa.bin',
  '7354323ff72a6132359c5aa76bf9904a.bin',
  '75a00aad5b506496a2644488064c39b4.bin',
  '7eba17d4ea5615e239c00e47d182e08a.bin',
  '91db8962e1a67a82cb5f9d2fd59ca9ff.bin',
  '96f125505ca736ff5d85ed33fa1c9b15.bin',
  '98d38858bcdcb327df21c3aee1ff40bc.bin',
  'a1f3ac69bc5030ecf1b351246986621a.bin',
  'b808c078b9786115200002b910a5a05c.bin',
  'd130caa1294885cc39ee0bc38037a0ee.bin',
  'df7bba1c415ec78a6d9e29d6c92f8a86.bin',
  'e1ec02725fcf3bce48e3131b1a6f2684.bin',
  'e8c7e887a3e46bfe41fc34a396184f79.bin',
  'eaa1aa078adc417afb112e145cb624f6.bin',
  'eadf8abf7738144e929a25e9eb7d8076.bin'],
 'Mirai': ['3d9487191dd4e712cbfb8f4dcf916a707f60c3fb23807d4c02fb941e216f951d.bin',
  'ac13002f74249e0eab2d

### Build 
[solved] Must run in `.py` file, cant run in `.ipynb`. Don't know why.
- 注意 trace folder 和 csv 的家族名稱不同，一個有 `-all` 一個沒有，兩者要一致否則會出 node 都是 0。
- 跑到 xorddos 的 ./trace/Dofloo/xxx 噴 FileNotFoundError: [WinError 2] 系統找不到指定的檔案。
    - f477b05826c46ac8a482e85d89b00de0.bin (有時候又OK)

In [147]:
# ! pip install graphviz

In [148]:
from cc_module import build

for family_name in sample_dct:
    if family_name != 'Dofloo': # 只建這個 family 的圖
        continue
    print(f'building family {family_name}')
    for s in sample_dct[family_name]:
        print(f'building sample {s}')
        try:
            # 建立圖譜並存檔
            build(family_name, s)
        except FileNotFoundError:
            print('[Error] FileNotFoundError')
    # [build(family_name, s) for s in sample_dct[family_name]] #　一行搞定版

building family Dofloo
building sample 0046a78514a658b9b7a4e29e8581f85b.bin
path: ./trace/Dofloo/0046a78514a658b9b7a4e29e8581f85b.bin
Nodes after reduction:  36
building sample 121ca5362f9bc3fff5258b757e4250e5.bin
path: ./trace/Dofloo/121ca5362f9bc3fff5258b757e4250e5.bin
Nodes after reduction:  35
building sample 26d45f2e2d360ff1707c734006878dcb.bin
path: ./trace/Dofloo/26d45f2e2d360ff1707c734006878dcb.bin
Nodes after reduction:  35
building sample 392214344b6d802dc4da644be260277e.bin
path: ./trace/Dofloo/392214344b6d802dc4da644be260277e.bin
Nodes after reduction:  36
building sample 44d98df44327179d1a11dce725d0fa4a.bin
path: ./trace/Dofloo/44d98df44327179d1a11dce725d0fa4a.bin
Nodes after reduction:  36
building sample 52ff8e32c390389f5bf18e0f0e2d9ada.bin
path: ./trace/Dofloo/52ff8e32c390389f5bf18e0f0e2d9ada.bin
Nodes after reduction:  25
building sample 54a11aacadee0ed46e76373cfb8ed296.bin
path: ./trace/Dofloo/54a11aacadee0ed46e76373cfb8ed296.bin
Nodes after reduction:  36
building sa

### Build 並回傳 ASG 資料結構
可以從 class AttackGraph 取得圖譜上的節點， asg.graph.keys()。

In [149]:
# 跑單一個圖
from cc_module import build

asg, g = build('Dofloo', '9a37fcc7eab08d59532bc5c66390bc30.bin')
type(g)

path: ./trace/Dofloo/9a37fcc7eab08d59532bc5c66390bc30.bin
Nodes after reduction:  36


graphviz.graphs.Digraph

In [150]:
names = [node.name for node in asg.graph.keys()]
set(names)

{'/dev/urandom',
 '/etc/init.d/boot.local',
 '/etc/rc.d/rc.local',
 '/etc/rc.local',
 '/proc/net/dev',
 '/proc/stat',
 '/sys/devices/system/cpu/online',
 '1529',
 '1530',
 '1531',
 '1532',
 '183.131.83.38:50050',
 'eth0',
 'eth1',
 'eth2',
 'eth3',
 'eth4',
 'eth5',
 'eth6',
 'eth7',
 'eth8',
 'eth9',
 'malware',
 'sed',
 'sh',
 'uname'}

### Nodes 統計紀錄


取得所有樣本的 Node，進行比對。

In [160]:
from cc_module import build
df = pd.read_csv("../C malware info/sample_info.csv")

# key: family -> nodes, value: freq counts of the node
familyNameList = df.family.unique()
# '{}' is not a factory!!
# https://stackoverflow.com/a/29015307
nodes_value_cnts = {key: {} for key in familyNameList}
# print("Initial nodes_value_cnts is: ", nodes_value_cnts)
for inx, row in df.iterrows():
    family_name = row.family
    sample_name = row.filename
    # print(family_name, sample_name)
    # if family_name not in ['Mirai', 'Tsunami']:
    #     continue
    asg, g = build(family_name, sample_name, save_file=False)
    nodes = set([node.name for node in asg.graph.keys()]) # nodes of the asg in this sample
    for n in nodes: # record the freq of this sample
        if isinstance(n, int): # if node is pid -> ignore
            continue
        try:
            int(n)
            continue
        except:
            pass
        family_dict = nodes_value_cnts[family_name]
        try:
            family_dict[n] += 1
            # print("nodes_value_cnts[", family_name, "][", n, "] +1=", family_dict[n])
        except KeyError:
            family_dict[n] = 1
            # print("nodes_value_cnts[",family_name, "][", n, "] init: ", family_dict[n])
    # print("Now nodes_value_cnts is: ", nodes_value_cnts)

path: ./trace/Xorddos/07c070b717a23453a2b71c7582d6a928.bin
Nodes after reduction:  1097
path: ./trace/Xorddos/0aefb67c01a24d05351b093455203fa2.bin
Nodes after reduction:  992
path: ./trace/Xorddos/0bc90c333f08237475a08c7158aba345.bin
Nodes after reduction:  991
path: ./trace/Xorddos/127ee9c007774e8f1101fb40ea05b42e.bin
Nodes after reduction:  1091
path: ./trace/Xorddos/16eb5d018d45024462dd63de470a8575.bin
Nodes after reduction:  1106
path: ./trace/Xorddos/24dca1b2097ad6c2e5d8b03d3022f010.bin
Nodes after reduction:  981
path: ./trace/Xorddos/39b38284c6ca30d09481fcb5dc7197aa.bin
Nodes after reduction:  1104
path: ./trace/Xorddos/7354323ff72a6132359c5aa76bf9904a.bin
Nodes after reduction:  976
path: ./trace/Xorddos/75a00aad5b506496a2644488064c39b4.bin
Nodes after reduction:  989
path: ./trace/Xorddos/7eba17d4ea5615e239c00e47d182e08a.bin
Nodes after reduction:  987
path: ./trace/Xorddos/91db8962e1a67a82cb5f9d2fd59ca9ff.bin
Nodes after reduction:  1106
path: ./trace/Xorddos/96f125505ca736ff

In [155]:
# copy & assert
Xorddos = nodes_value_cnts['Xorddos']
Mirai = nodes_value_cnts['Mirai']
Mirai == Xorddos

{'Xorddos': {},
 'Mirai': {'8.8.8.8:53': 3,
  '192.168.1.2:38273': 1,
  'malware': 3,
  '0.0.0.0:38273': 1,
  '5.182.210.244:38273': 1,
  '/proc/net/tcp': 1,
  '255.255.255.255:3074': 1,
  '111.240.131.42:23': 1,
  '/dev/misc/watchdog': 2,
  '/dev/watchdog': 2,
  '/bin/watchdog': 2,
  '/etc/watchdog': 2,
  '/dev/watchdog0': 2,
  '/proc/self/exe': 1,
  '136.144.41.60:3074': 1,
  '/proc/1431/cmdline': 1,
  '/proc/1801/cmdline': 1,
  '/proc/1799/cmdline': 1,
  '/proc/1813/cmdline': 1,
  '219.244.162.21:23': 1,
  '195.236.162.21:80': 1,
  '/proc/1795/cmdline': 1,
  '/proc/1422/cmdline': 1,
  '/proc/1811/cmdline': 1,
  '/proc/1803/cmdline': 1,
  '/etc/default/watchdog': 1,
  '/proc/1086/cmdline': 1,
  '/proc/1882/cmdline': 1,
  '/proc/1797/cmdline': 1,
  '/proc/1798/cmdline': 1,
  '/proc/1793/cmdline': 1,
  '/proc/1505/cmdline': 1,
  '/proc/1883/cmdline': 1,
  '/proc/1385/cmdline': 1,
  '/proc/1384/cmdline': 1,
  '/proc/1399/cmdline': 1,
  '/proc/1881/cmdline': 1,
  '/proc/1889/cmdline': 1,

In [166]:
# save nodes_value_cnts
with open('nodes_value_cnts.pkl', 'wb') as outp:
    pickle.dump(nodes_value_cnts, outp, pickle.HIGHEST_PROTOCOL)

In [165]:
# analysis the nodes_value_cnts
with open('./nodes_value_cnts_analysis.txt','w') as file:
    family_name_lst = nodes_value_cnts.keys()
    print(family_name_lst)
    for f in family_name_lst:
        file.write(f"# Family {f}\n")
        nodes = nodes_value_cnts[f]
        for n in list(nodes.keys()):
            try:
                int(n)
                nodes.pop(n)
            except:
                pass
        sorted_nodes = dict(sorted(nodes.items(), key=lambda item: item[1], reverse=True))
        for k,v in sorted_nodes.items():
            file.write(f'{k}: {v}\n')
            # if v > 1:
            #     print(f'{k}: {v}')
        file.write('\n\n')

        # 以 sort 過的 dict 取代原有的
        nodes_value_cnts[f] = sorted_nodes

dict_keys(['Xorddos', 'Mirai', 'Dofloo', 'Tsunami'])


In [158]:
edges = graph.edges
for nodes, edge in edges.items():
    print(nodes.name)

NameError: name 'graph' is not defined